# When worlds collide: optimization under uncertainty

In the previous optimization notebook ([freyberg opt 1](../part2_08_opt/freyberg_opt_1.ipynb)), we saw how we can use the PEST interface to implement formal constrained management optimization.  And it was awesome!

But what about all those other notebooks where we droned on and on about prediction uncertainty and parameter estimation/data assimilation?  Was that all for nothing?! 

No!  It wasn't.  In fact, in the same way that predictions are uncertain, constraints based on simulation results are also uncertain.  In the previous optimization notebook we worked through an example in which future surface-water/groundwater exchange flux was used as a constraint to avoid an unwanted outcome (too little flux to sustain ecological flows). In that exercise we determined the (simulated!) optimal pumping rate that maximized abstraction whilst also respecting the minimum ecological flow rate - for a _single set of parameter values_! But, as we have seen _ad infinitum_ throughout these tutorials, model outputs are uncertain. So it's only natural to think "how can we combine the uncertainty analysis concepts with management optimization?".  This is referred to as __optimization under uncertainty__ and the crux of the deal is the concept of "chance constraints".  

Most optimization algorithms require that constraints be assigned a single "right-hand side" (RHS) value - the value not to violate. (The value that, if exceeded, is a "bad thing")  But uncertainty analysis gives us a range (or statistical distribution) of possible constraint/prediction values.  How can we rectify this problem?  By accounting for the probability distribution of the model output which is the constraint. 

A simple way to think of this is as "shifting the constraint to accommodate for uncertainty". Taking our minimum ecological flow constraint as an example: we determined optimal pumping rates that ensure that sw-gw exchange rates were always positive (i.e. no stream depletion). But if we determine the uncertainty of sw-gw exchange rate with the optimal pumping rates, we might find that the probability distribution includes negative exchange rates (i.e. stream depletion) - which we don't want. However, we can now determine by how much we need to shift the constraint target value to ensure that it is respected to a desired level of confidence. 

This leads us to a concept of "risk shifting". In this context, "risk" (aka reliability) becomes a simple scalar algorithmic control that ranges from 0.0 to 1.0.  

A risk value of 0.5 is called "risk neutral" and it implies that 50% of the mass of a constraint probability density function is on either side of the value that corresponds to risk = 0.5 (think of the mean of a normal distribution:  at the mean value, half of the distribution is on either side). Conceptually, the optimization we undertook with a single parameter set in the previous notebook took a risk neutral stance. Implicitly, the initial parameter values reflect the mean of the parameter probability distribution. 

A risk of 0.95 implies we want to be 95% sure the constraint will not be violated - referred to as risk averse.  A risk averse stance implies we will have to accept a sub optimal objective function to be 95% sure.  The other side of the distribution is referred to as "risk tolerant" and it implies a decreasingly small chance that the constraint will actually be satisfied (danger zone!).  

Why did we say "actually be satisfied" above?  Well, just like predictions, we don't know what the "true" or "real world" value of the model-based constraints is (if we did, we wouldn't need to be modeling at all!).  So we don't know what the "true" constraint will take because its something we cant or haven't observed.  

Ok, enough words.  Let's see how this works in practice.  

Note that to run this notebook, you will have needed to run both the previous optimization notebook ([freyberg opt 1](../part2_08_opt/freyberg_opt_1.ipynb)), as well as the [freyberg ies 1](../part2_06_ies/freyberg_ies_1_basics.ipynb) notebook.

### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see ["freyberg intro to model"](../part0_02_intro_to_freyberg_model/intro_freyberg_model.ipynb) notebook), and the high-dimensional PEST dataset prepared in the ["freyberg pstfrom pest setup"](../part2_01_pstfrom_pest_setup/freyberg_pstfrom_pest_setup.ipynb)" and ["observation and weights"](../part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb) notebooks. 

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 10}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd

To maintain continuity in the series of tutorials, we we use the PEST-dataset prepared in the ["observation and weights"](../part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb) tutorial. Run the next cell to copy the necessary files across. Note that if you will need to run the previous notebooks in the correct order beforehand.

Specify the path to the PEST dataset template folder. Recall that we will prepare our PEST dataset files in this folder, keeping them separate from the original model files. Then copy across pre-prepared model and PEST files:

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template_chance')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_08_opt/freyberg_opt_1.ipynb' notebook")

shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')

In [ ]:
pst = pyemu.Pst(pst_path)

# Calculating Constraint Uncertainty


PESTPP-OPT incorporates model-based constraint uncertainties in [three ways](https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md#813-calculation-of-uncertainty): 
- by supplying model-based constraint "weights" in the PEST control file as standard deviations, 
- by using linear method and
- by using "stack-based" constraint uncertainty.

As is implicit in the description, the first option assumes the user has pre-calculated the constraint standard deviation and is assigning it explicitly. This is the most rudimentary approach. 

The second option relies on FOSM (or linear analysis) methods to calculate constraint uncertainties. It is also affected by the same assumptions - that there is a linear relation between decision variables and model-outputs. As we have seen in previous notebooks on PESTPP-GLM and FOSM, this approach requires that the user supply the prior parameter uncertainty, measurement noise. PESTPP-OPT also requires a Jacobian matrix with calibration-adjustable parameter sensitivities as well as the sensitivities to the chance constrain model outputs. These can be provided by the user, or PESTPP-OPT can calculate them as part of the optimization run.

The third, stack-based, approach is the most robust as it avoids the assumption of linearity between parameters and model-output constraints. The "stack" is simply another word for a parameter ensemble. PESTPP-OPT "evaluates" the stack by running the model once for each parameter realization, with the current decision variable values. From these runs, PESTPP-OPT obtains an observation stack (i.e. an ensemble) that contains the stochastic sample of model-outputs (i.e observations) for the constraint. This sample is then used to characterize the constraints' probability distribution. 


## Stacks

So mechanically, how do we come up with this constraint probability distribution function (PDF)?  We saw previously in the PESTPP-IES notebook that we had to run the posterior parameter ensemble to yield a predictive PDF.  Well its no different here:  We will grab that PESTPP-IES posterior parameter ensemble (and manipulate it a little to remove decision variables) and then identify that ensemble as a "stack" of parameter realizations that can be run through the model to yield constraint PDFs.  Easy as!  

Beware though: including a stack in the optimization means we need to evaluate the stack at least once (see "coupling" below) which means we need to queue up and run the stack along with the response matrix perturbation runs from before...lucky for you PESTPP-OPT does this automagically!

In [ ]:
# check that the pestpp-ies directory exists and that the posterior parameter ensemble exists
ies_dir = os.path.join("..","part2_06_ies","master_ies_1")
if not os.path.exists(ies_dir):
    raise Exception("you need to run the 'part2_06_ies/freyberg_ies_1_basics.ipynb' notebook")

In [ ]:
pe_files = [f for f in os.listdir(ies_dir) if f.endswith(".par.csv") and f.startswith("freyberg_mf6") and "mean" not in f]
pe_files.sort()
pe_files

Now load the parameter ensemble from the last iteration of PESTPP-IES:

In [ ]:
last_iter = max([int(i.split('.')[-3]) for i in pe_files if i.split('.')[-3].isdigit()])
last_iter_pe_file = [i for i in pe_files if i.split('.')[-3]==str(last_iter)][0]
last_iter_pe_file

In [ ]:
pe = pd.read_csv(os.path.join(ies_dir,last_iter_pe_file),index_col=0)
pe.head()

Set decision variable parameter values in the _parameter ensemble_ to a fixed "initial value":

In [ ]:
par = pst.parameter_data
#par.loc[par.partrans=="fixed","partrans"] = "none"
wpar = par.loc[par.parnme.str.contains("wel") & par.parnme.str.contains("cn"),"parnme"]
pe.loc[:,wpar.values] = 1.0

Write the new parameter ensemble "stack" to a CSV:

In [ ]:
pe.to_csv(os.path.join(t_d,"par_stack.csv"))

Update the pestpp options to inform PESTPP-OPT of the stack file:

In [ ]:
pst.pestpp_options["opt_par_stack"] = "par_stack.csv"
pst.pestpp_options["opt_risk"] = 0.95

Some convenience stuff for later:

In [ ]:
# copy so we can access the original values later on
obs_org = pst.observation_data.copy()

obs = pst.observation_data

Re-write the control file:

In [ ]:
pst.noptmax = 1
pst.write(pst_path,version=2)

## An aside on "coupling": interaction among decision variables, parameters, and constraints
This concept is important and a bit of tricky business, so we are explaining it in two different ways.

### First coupling interpretation:

So we are trying to jam these two very different decision-support concepts (and in fact fields of study) together into a single algorithm.  Its awesome but as you might expect there are "knock-on" effects.  The biggest one (besides increased computational burden) is the concept of "coupling".  Coupling refers to interaction between the uncertainty process (in this case monte carlo "stack evaluation") and the optimization process (adjusting groundwater extraction rates).  Let's talk about linearity...

What do we mean by "linear"?  Well, in the uncertainty process, we mean that the relation between parameters and model outputs of interest is constant across the range of parameter values (and combinations) - think of the Jacobian matrix having the same values no matter where it is calculated in parameter space.

In the optimization process, its a similar definition of \"linear\": the relation between decision variables and outputs of interest is also constant across decision variable space.

But wait! there is more - we also have to now make sure that as the parameters change, they don't change the relation between decision variables and outputs of interest (and vis-a-versa) - wat?!  We can think of these as "cross terms" between the uncertainty and optimization processes.  Think of it this way:  as we start to extract more groundwater and the cone of depression grows, we are "activating" the impact of more and more parameters in the relation between parameters and simulated drawdown because previously insensitive parameters (think HK distall to the location of pumping) are now contributing to the uncertainty in the simulated drawdown.  headache yet?

So what does this mean?  Well, if we expect a \"strong\" coupling, we need to account for this in the optimal solution, which means we might need to re-evaluate the stack more frequently in the optimization solution process - ugh, more runs!,

For this notebook (and this freyberg problem) coupling is not really a problem because the model (and the uncertainty and optimization problems) are pretty linear.  But in the case of transport modeling, coupling is much stronger...

 

### Second coupling interpretation:

As you may recall from previous tutorials (e.g. ["intro to GLM"](../part1_06_glm_response_surface/freyberg_glm_response_surface.ipynb) in Part1, and the ["freyberg_glm_1"](../part2_04_glm/freyberg_glm_1.ipynb) and ["freyberg_glm_2"](../part2_04_glm/freyberg_glm_1.ipynb) notebooks in Part2), during _parameter estimation_ we are efectively assessing the relation between model parameters and model outputs (i.e. target observations). As we saw in those notebooks, as we move through parameter space (i.e. as we change parameter values), if the model is not linear (which it never is), then the relation between parameter and observations will change. This is why we undertake parameter estimation iteratively - to re-calculate the parameter-to-observation change relations as we progress through parameter space.

In an optimization under uncertainty analysis, the same concept applies, with a slight twist. In this case, we are evaluating two fundamental relations:

1.  __decision variable relation__: the relation between _decision variables_ (parameters) and model-derived _constraints_ (observations).
2. __parameter relation__: the relation between _uncertain parameters_ and model-derived _constraints_ (observations).

Each of these relations may change as we move through decision variable/uncertain parameter space. Added to this, the _decision variable relation_ may also change as we move through the _uncertain parameter_ space (and vice versa). So, if changes in the _decision variable_ values influence the _parameter relation_, or changes in the _uncertain parameters_ influence the _decision variable relation_, then this interaction should ideally be considered during the optimization under uncertainty analysis - as it may influence the optimal risk-based solution. We refer to this interaction as "__coupling__".

If no coupling exists, then no matter what the _decision variable_ values are, the constraint uncertainties calculated at any point in the decision variable space are always valid. This means that uncertainties need only be quantified once. For a FOSM-based approach, this means filling a Jacobian by running the model once for every uncertain parameter; for a stack-based approach it means running each realisation in the parameter stack.

However, if coupling does exist (which invariably is the case...), then the relation between _uncertain parameters_ and constraints need to be re-evaluated as we move through _decision variable_ space. As you might surmise from the previous paragraph, this can clock up alot more computational time (e.g. FOSM-based requires re-filling the Jacobian; stack-based requires re-running the parameter stack).

The choice of how and when to (re-)evaluate is very problem specific and depends on many factors like nonlinearity of the problem, model run time, and available computational resources. PESTPP-OPT provides a great deal of flexibility here. The `opt_recalc_chance_every()` control variable allows the user to specify the frequency (e.g. number of iterations) at which the chance constraints are re-assessed.

For the purposes of this tutorial, we are only going to assess chance constraints once at the start of the SLP process. As mentioned, when using stacks, PESTPP-OPT does this automagically for you. (If using a FOSM-based approach, you could also supply an existing Jacobian matrix; see the [user manual](https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md#84-summary-of-control-variables) for all options).


# Attention!

You must specify the number which is adequate for ***your*** machine! Make sure to assign an appropriate value for the following `num_workers` variable:

In [ ]:
num_workers = 15 # update according to your available resources!

Then specify the folder in which the PEST manager will run and record outcomes. It should be different from the `t_d` folder. 

In [ ]:
m_d = os.path.join('master_opt_2')

The following cell deploys the PEST agents and manager and then starts the run using `pestpp-opt`. Run it by pressing `shift+enter`.

If you wish to see the outputs in real-time, switch over to the terminal window (the one which you used to launch the `jupyter notebook`). There you should see `pestpp-opt`'s progress. 

If you open the tutorial folder, you should also see a bunch of new folders there named `worker_0`, `worker_1`, etc. These are the agent folders. `pyemu` will remove them when PEST finishes running.

This run should take a while to complete (depending on the number of workers and the speed of your machine). If you get an error, make sure that your firewall or antivirus software is not blocking `pestpp-opt` from communicating with the agents (this is a common problem!).

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-opt","freyberg_mf6.pst",num_workers=num_workers,worker_root=".",
                           master_dir=m_d)

### Processing PESTPP-OPT


Let's first get a few lists of observation names to make postprocessing easier:

In [ ]:
obs = obs_org.loc[obs_org.weight > 0,:].copy()
wel_constraint_names = obs.loc[obs.obsnme.str.contains("inc") & obs.obsnme.str.contains("wel"),"obsnme"]
swgw_constraint_names = obs.loc[obs.obsnme.str.contains("inc") & obs.obsnme.str.contains("sfr"),"obsnme"]

This time we have a whole bunch of *.rei residual files. (See the [manual](https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md#83-pestpp-opt-output-files) for descriptions of each.) 

In [ ]:
[f for f in os.listdir(m_d) if f.endswith(".rei")]

Now we also have "chance" files, which, as the name implies, are residual files that represent the estimated and simulated observation quantities with the chance/risk offsets included.  Let's compare:

In [ ]:
swgw_rhs = obs.loc[swgw_constraint_names,"obsval"].max()
wel_rhs = obs.loc[wel_constraint_names,"obsval"].max()
chance_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_mf6.2.est+chance.rei"))
est_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_mf6.2.est.rei"))
#constraints = swgw_constraint_names.tolist()
#constraints.extend(wel_constraint_names)
fig,ax = plt.subplots(1,1,figsize=(10,3))
est_df.loc[swgw_constraint_names,"chance+estimated"] = chance_df.loc[swgw_constraint_names,"modelled"]
est_df.loc[:,"estimated"] = est_df.modelled.values
est_df.loc[swgw_constraint_names,["estimated","chance+estimated"]].plot(ax=ax,kind="bar")
ax.plot(ax.get_xlim(),[swgw_rhs,swgw_rhs],"k--");

Here we see the cost of uncertainty:  we have to leave a larger amount groundwater in the system so that be can be sure (at 95% confidence) that the sw-gw flux will remain 0.0 or negative (the orange bars are right at 0.0 for most stress periods, showing the precision of the optimization solver)

Now let's tie it all together. As in the last notebook, brace yourself for some hackery so that we can make a pretty plot:

Access the optimal decision variable (pumping rate) parameter values:

In [ ]:
# get dec variable parameter names
wpar = par.loc[par.pargp=="decvars",:]
future_wpar_names = wpar.parnme
# get pestpp-opt obtained optimal values:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_mf6.par"))
par_df.loc[future_wpar_names,:].head()

In [ ]:
# put initial and optimal dec variable parameter values into the same dataframe
wpar = wpar.loc[future_wpar_names,:].copy()
wpar.loc[:,"kij"] = wpar.apply(lambda x: (x.idx0,x.idx1,x.idx2),axis=1)
wpar.loc[:,"optimal"] = par_df.loc[wpar.parnme,"parval1"]
wpar.head()

In [ ]:
# identify unique wells
inst_vals = wpar.inst.unique()
inst_vals.sort()
inst_vals

In [ ]:
# organize colors for bar plot
colors = ["r","g","b","c","m","y","0.5"]
vals = {}
for inst in inst_vals:
    ipar = wpar.loc[wpar.inst==inst,:].copy()
    ipar.sort_values(by="kij",inplace=True)
    ipar.index = ipar.kij
    #ipar.optimal.plot(ax=ax,kind="bar",color=colors)
    vals[inst] = ipar.optimal

In [ ]:
# plot this beast
fig,axes = plt.subplots(2,1,figsize=(12,6))
colors = ["r","g","b","c","m","y","0.5"]
df = pd.DataFrame(vals).T
df.plot(ax=axes[0],kind="bar",color=colors)
axes[0].set_ylim(0,9)
nconst = len(wel_constraint_names)-1
axes[1].plot(np.arange(nconst),est_df.loc[wel_constraint_names,"modelled"].values[1:],"b",lw=1.5)
axes[1].plot(axes[1].get_xlim(),[wel_rhs,wel_rhs],"b--",lw=3.5)
axes[1].fill_between(np.arange(nconst),np.zeros(nconst) + wel_rhs,
                     est_df.loc[wel_constraint_names,"modelled"].values[1:],facecolor="b",alpha=0.5)
axt = plt.twinx(axes[1])
axt.plot(np.arange(nconst),est_df.loc[swgw_constraint_names,"modelled"].values[1:],"m",lw=1.5)
axt.fill_between(np.arange(nconst),np.zeros(nconst) + swgw_rhs,
                     est_df.loc[swgw_constraint_names,"modelled"].values[1:],facecolor="m",alpha=0.5)
axes[1].set_xticklabels(inst_vals)
axes[0].set_xlim(0,12)
axes[1].set_xticks(np.arange(len(wel_constraint_names)-1))
axes[1].set_xlim(0,12)
axt.plot(axes[1].get_xlim(),[swgw_rhs,swgw_rhs],"m--",lw=3.5)
axes[1].set_ylim(-10000,0)
axt.set_ylim(-700,800)
axes[0].set_title("Decision Variables",loc="left")
axes[1].set_title("Constraints",loc="left")
axes[1].grid()
[i.set_color("b") for i in axes[1].get_yticklabels()]
[i.set_color("m") for i in axt.get_yticklabels()]
lb = axes[1].set_ylabel("groundwater extraction rate")
lb.set_color('b')
lb = axt.set_ylabel("sw-gw exchange rate")
lb.set_color('m')
plt.tight_layout();

The plot in the cell above shows:
 - (top) Optimal pumping rates at each well during future stress periods. Wells are distinguished by different coloured bars. 
 - (bottom) Total groundwater extraction rate (blue line) and sw-gw exchange rate (purple line). Constraints are identified by the respective blue and purple horizontal dashed-lines. 

Again, we see the cost of uncertainty: we are pumping much less water compared to the risk-neutral case (in the previous tutorial) so that we can leave that groundwater to discharge to the surface-water system. e.g. we are more certain that the management scheme will respect the constraint! #reliability

## Reusing previous results 

Results shown in the plot above are for a risk-averse stance that at 95% level of confidence. What happens if we shift our risk stance? How does that change the "optimal" groundwater extraction rate?

Assuming (big assumption!) we can rely on the estimated values from the solver, we can do some trickery to skip any additional model runs while we explore the additional problem formulations.  

As is described in the [PEST++ user manual](https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md#83-pestpp-opt-output-files):

> PESTPP-OPT offers functionality for solving the chance-constrained SLP problem without the requirement for any model runs. If a user activates the base_jacobian(), hotstart_resfile() and opt_skip_final() options while setting the NOPTMAX control variable to 1, then PESTPP-OPT will not undertake any model runs at all. Instead, it will solve the chance-constrained linear programming problem specified in the control file, report optimal decision variable values and the final objective function, and then cease execution. This can be a useful strategy for exploring the implications of changing decision variable bounds, constraints, risk and/or any of the factors affecting chance constraints. The latter can include prior parameter uncertainties, and the number of observations (and their weights) used to condition parameters.

This means we can loop through solving the sequential linear programming (SLP) problem, simply adjusting values in the PEST control file and not requiring any additional model runs.

Start by making copies of files needed for running PESTPP-OPT in restart mode:

In [ ]:
# the jacobian
shutil.copy2(os.path.join(m_d,"freyberg_mf6.1.jcb"),os.path.join(m_d,"restart.jcb"))
pst.pestpp_options["base_jacobian"] = "restart.jcb"
# the residual file
shutil.copy2(os.path.join(m_d,"freyberg_mf6.1.jcb.rei"),os.path.join(m_d,"restart.res"))
# the res file
pst.pestpp_options["hotstart_resfile"] = "restart.res"
shutil.copy2(os.path.join(m_d,"freyberg_mf6.1.obs_stack.csv"),os.path.join(m_d,"obs_stack.csv"))

In [ ]:
# specify the observation stack; these are model outputs for each model run with parameters in the parameter stack
pst.pestpp_options["opt_obs_stack"] = "obs_stack.csv"
# dorp parameter stack, cause we aint running them again
pst.pestpp_options.pop("opt_par_stack",None)
# specify to skip final; again, no models are being run
pst.pestpp_options["opt_skip_final"] = True

In [ ]:
# as usual re-write the control file
pst.control_data.noptmax = 1
pst.write(os.path.join(m_d,"freyberg_mf6_restart.pst"),version=2)

In [ ]:
# run pestpp-opt once to check
pyemu.os_utils.run("pestpp-opt freyberg_mf6_restart.pst",cwd=m_d)

OK, here we go. Let's do something fun:  blast over a full range of risk values and see how the objective function changes. Code heavy section coming up:

In [ ]:
def scrape_rec():
    """Function to scrape the *.rec file and extract best objective fucntion value."""
    with open(os.path.join(m_d,"freyberg_mf6_restart.rec"),'r') as f:
        for line in f:
            if "---  best objective function value:" in line:
                #print(line)
                obj_val = float(line.strip().split()[-1])
                break
    return obj_val

In [ ]:
cwname = "oname:cum_otype:lst_usecol:wel_totim:4383.5"
cw_vals = []
pst.observation_data.loc[cwname,"obgnme"] = "less_than"
pst.observation_data.loc[cwname,"obsval"] = 1.0e+10
pst.observation_data.loc[cwname,"weight"] = 1.0

Now, we will loop through 100 "risk" values, solve the SLP and record the outcomes (this next cell may take a few minutes):

In [ ]:
risk_vals = np.linspace(0.001,0.999,100)
# reord objective function values
obj_vals = []
for risk_val in risk_vals:
    # update the risk value
    pst.pestpp_options["opt_risk"] = risk_val
    # rewrite control file
    pst.write(os.path.join(m_d,"freyberg_mf6_restart.pst"))
    #run pest opt with "zero run" options
    pyemu.os_utils.run("pestpp-opt freyberg_mf6_restart.pst",cwd=m_d)
    # get obj values form the rec file
    obj_vals.append(scrape_rec())
    # get residuals from the res file
    df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_mf6_restart.1.est.rei"))
    cw_vals.append(df.loc[cwname,"modelled"])

OK, let's plot that up:

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
ax.plot(risk_vals,obj_vals)
ax.grid()
ax.set_xlabel("risk")
ax.set_ylabel("objective function")
ax.set_xticks(np.arange(0,1.1,0.1))
axt = plt.twinx(ax)
axt.plot(risk_vals,np.array(cw_vals)*-1,alpha=0.0)
axt.set_ylabel("cumulative groundwater extracted ($L^3$)")
plt.tight_layout()
plt.show()

So that is a million dollar plot!  We are seeing the optimal solution to the constrained groundwater management problem (right y-axis) across varying risk stances (x-axis). And we can assign cost of uncertainty in terms of volume of extracted by differencing the maximum extraction from the minimum extraction across the range of risk values.  

This trade-off between risk and objective function is called a "pareto frontier" or "pareto front" - we will cover these trade-off surfaces in more detail in the MOU notebooks, but for now, it is valuable to think about the relation between the desire to produce more groundwater for economic use, while managing the risk of causing an unwanted environmental outcome, namely decreased groundwater discharge to surface-water. As we seek more assurance that the unwanted outcome wont happen (by increasing the "risk" variable, which can also be thought of as increasing reliability), we in turn must leave more groundwater inplace to potentially supply the surface-water system during critical low-flow periods. #optimizationunderuncertainty